<a href="https://colab.research.google.com/github/cam2149/MachineLearningV/blob/main/CNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Equipo**

- Nicolás Colmenares

- Carlos Martinez

1. Implementación de una Red Convolucional (CNN) adaptada a series temporales.- 1 pts

  -  .

  - .

  - .

**Situación:**
Una ciudad enfrenta un aumento significativo de casos de dengue, con una tasa de incidencia que supera el promedio nacional.
La anticipación de brotes es crucial para implementar medidas preventivas y reducir la propagación de la enfermedad.

**Objetivo:**
Desarrollar un modelo predictivo utilizando redes neuronales para pronosticar futuros brotes de dengue en cada barrio de la ciudad.
Utilizar una base de datos histórica de casos de dengue desde 2015 hasta 2022 para entrenar el modelo.
Anticiparse a los brotes con al menos 3 semanas de anticipación.

**Finalidad:**
Permitir a las autoridades de salud pública tomar acciones oportunas, como:
Preparar a las instituciones prestadoras de salud (IPS).
Gestionar recursos (carros fumigadores, limpieza de sumideros).
Capacitar a la comunidad.

*   Red Convolucional (CNN) adaptada a series temporales.
*   .
*   .

# 0. Configuraciones de Colab

Mover Kaggle.json a la ubicación correcta después de subirlo

In [ ]:
#Estas líneas son comandos de shell que se ejecutan dentro del Jupyter notebook. Se usan para configurar las credenciales de la API de Kaggle, que son necesarias para descargar conjuntos de datos (datasets) desde Kaggle.

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!rm -rf /content/kaggle/output
!rm -rf /content/kaggle/input

Descargar dataset de la competencia

In [ ]:
!kaggle competitions download -c aa-v-2025-i-pronosticos-nn-rnn-cnn

In [ ]:
!mkdir -p /content/kaggle/output
!mkdir -p /content/kaggle/input

In [ ]:
!mv aa-v-2025-i-pronosticos-nn-rnn-cnn.zip /content/kaggle/input

In [ ]:
!unzip /content/kaggle/input/aa-v-2025-i-pronosticos-nn-rnn-cnn.zip -d /content/kaggle/input/

In [ ]:
#/kaggle/input
import os
for dirname, _, filenames in os.walk('/content/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# 1. Imports

In [ ]:
!pip install altair

In [ ]:
import itertools
import datetime
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import altair as alt
import matplotlib as mpl
import matplotlib.pyplot as plt   # data visualization
import seaborn as sns
import statsmodels.api as sm
import scipy
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset
from sklearn.metrics import mean_squared_error, mean_absolute_error, recall_score, roc_auc_score, accuracy_score, roc_curve
from datetime import datetime, timedelta
from tqdm import tqdm


In [ ]:
#Printing library versions
print('Pandas:', pd.__version__)
print('Statsmodels:', sm.__version__)
print('Scipy:', scipy.__version__)
print('Matplotlib:', mpl.__version__)
print('Seaborn:', sns.__version__)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# 2. Configs

In [ ]:
config = {
    "TRAIN_DIR": '/content/kaggle/input/df_train.parquet',
    "TEST_DIR": '/content/kaggle/input/df_test.parquet',
    "SUBMISSION_DIR": '/content/sample_submission.csv'
}

# Exploración

## Diccionario

train.parquet - El conjunto de datos de entrenamiento
test.parquet - El conjunto de datos de prueba
sample_submission.csv - un ejemplo de un archivo a someter en la competencia

| **Variable**         | **Descripción**                                                                                      |
|-----------------------|------------------------------------------------------------------------------------------------------|
| id_bar               | identificador único del barrio                                                                      |
| anio                 | Año de ocurrencia                                                                                   |
| semana               | Semana de ocurrencia                                                                               |
| Estrato              | Estrato socioeconómico del barrio                                                                   |
| area_barrio          | Área del barrio en km²                                                                             |
| dengue               | Conteo de casos de dengue                                                                          |
| concentraciones      | Cantidad de visitas e intervención a lugares de concentración humana (Instituciones)                |
| vivienda             | Conteo de las visitas a viviendas a revisión y control de criaderos                                 |
| equipesado           | Conteo de las fumigaciones con Maquinaria Pesada                                                   |
| sumideros            | Conteo de las intervenciones a los sumideros                                                       |
| maquina              | Conteo de las fumigaciones con motomochila                                                         |
| lluvia_mean          | Lluvia promedio en la semana i                                                                     |
| lluvia_var           | Varianza de la lluvia en la semana i                                                               |
| lluvia_max           | Lluvia máxima en la semana i                                                                       |
| lluvia_min           | Lluvia mínima en la semana i                                                                       |
| temperatura_mean     | Temperatura promedio en la semana i                                                                |
| temperatura_var      | Varianza de la temperatura en la semana i                                                          |
| temperatura_max      | Temperatura máxima en la semana i                                                                  |
| temperatura_min      | Temperatura mínima en la semana i                                                                  |


## Lectura del dataset de entrenamiento

In [ ]:
#Esta celda tiene como objetivo leer los datos de entrenamiento desde un archivo Parquet y mostrar información básica sobre ellos
try:
    train_df = pd.read_parquet(config["TRAIN_DIR"]).iloc[:,1:]
    test_df = pd.read_parquet(config["TEST_DIR"]).iloc[:,1:]
    print(train_df.describe())
    print(train_df.info())
    print(train_df.shape)

    print(test_df.describe())
    print(test_df.info())
    print(test_df.shape)
except FileNotFoundError:
    print("Error: 'series_train.parquet' not found. Please make sure the file exists in the current directory or provide the correct path.")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
# Generar la columna 'fecha'
def get_last_day_of_week(year, week):
  first_day_of_year = datetime(int(year), 1, 1)
  return first_day_of_year + timedelta(days=(int(week)-1) * 7)

In [ ]:
 # Aplicar la función para generar la columna de fecha
try:
  train_df['fecha'] = train_df.apply(lambda row: get_last_day_of_week(int(row['anio']), int(row['semana'])), axis=1)
  test_df['fecha'] = test_df.apply(lambda row: get_last_day_of_week(int(row['anio']), int(row['semana'])), axis=1)
  # Establecer fecha como índice
  train_df = train_df.set_index('fecha')
  test_df = test_df.set_index('fecha')
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# Función para preparar las secuencias de datos para el modelo CNN
def preparar_sequences(df, sequence_length=12, forecast_horizon=3, target_col='dengue'):

    # Definir las columnas de características (todas excepto la variable objetivo)
    feature_cols = [col for col in df.columns if col not in ['id_bar', 'anio', 'semana', target_col]]

    # Normalizar las características
    scaler = StandardScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(df[feature_cols]),
                             columns=feature_cols,
                             index=df.index)

    # Agregar la variable objetivo
    if target_col in df.columns:
        df_scaled[target_col] = df[target_col]

    # Crear secuencias
    X, y = [], []

    # Agrupar por barrio (id_bar)
    for barrio_id in df['id_bar'].unique():
        # Filtrar datos por barrio
        barrio_data = df[df['id_bar'] == barrio_id]

        # Crear secuencias para este barrio
        for i in range(len(barrio_data) - sequence_length - forecast_horizon + 1):
            # Secuencia de entrada
            sequence = df_scaled.loc[barrio_data.index[i:i+sequence_length], feature_cols].values

            # Valor objetivo (casos de dengue después del horizonte de pronóstico)
            if target_col in df.columns:
                target = barrio_data.iloc[i+sequence_length+forecast_horizon-1][target_col]

                X.append(sequence)
                y.append(target)

    if target_col in df.columns:
        return np.array(X), np.array(y), feature_cols
    else:
        # Para datos de prueba donde no hay objetivo
        return np.array(X), None, feature_cols



In [ ]:
# Clase para el modelo CNN
class DengueForecastCNN(nn.Module):
    """
    Modelo CNN para pronóstico de brotes de dengue.
    """
    def __init__(self, input_channels, sequence_length, num_features):
        super(DengueForecastCNN, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=input_channels,
                               out_channels=64,
                               kernel_size=3,
                               padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv1d(in_channels=64,
                               out_channels=128,
                               kernel_size=3,
                               padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

        # Calcular tamaño de salida después de las capas convolucionales
        conv_output_size = sequence_length // 4 * 128

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(conv_output_size, 100)
        self.relu3 = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(100, 1)

    def forward(self, x):
        # Entrada: [batch_size, sequence_length, num_features]
        # Necesitamos transponer para Conv1d: [batch_size, num_features, sequence_length]
        x = x.permute(0, 2, 1)

        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))

        x = self.flatten(x)
        x = self.dropout(self.relu3(self.fc1(x)))
        x = self.fc2(x)

        return x.squeeze()



In [ ]:
# Función para entrenar el modelo
def entrenar_modelo(X_train, y_train, X_val, y_val, model, optimizer, criterion, epochs, batch_size=32):

    # Convertir a tensores
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.FloatTensor(y_train)
    X_val_tensor = torch.FloatTensor(X_val)
    y_val_tensor = torch.FloatTensor(y_val)

    # Crear conjuntos de datos
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Listas para almacenar pérdidas
    train_losses = []
    val_losses = []

    # Entrenamiento del modelo
    for epoch in range(epochs):
        model.train()
        train_loss = 0

        for X_batch, y_batch in train_loader:
            # Forward pass
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

            # Backward y optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Calcular pérdida promedio de entrenamiento
        train_loss = train_loss / len(train_loader)
        train_losses.append(train_loss)

        # Validación
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor)
            val_loss = criterion(val_outputs, y_val_tensor).item()
            val_losses.append(val_loss)

        # Imprimir progreso cada 10 épocas
        if (epoch + 1) % 10 == 0:
            print(f'Época {epoch+1}/{epochs}, Pérdida de entrenamiento: {train_loss:.4f}, Pérdida de validación: {val_loss:.4f}')

    return model, train_losses, val_losses



In [ ]:
# Función para evaluar el modelo
def evaluar_modelo(model, X_test, y_test):

    model.eval()

    # Convertir a tensor
    X_test_tensor = torch.FloatTensor(X_test)

    # Predicciones
    with torch.no_grad():
        predictions = model(X_test_tensor).numpy()

    # Calcular métricas
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)

    # Calcular accuracy (consideramos que predice correctamente si la diferencia es menor a 0.5)
    accuracy = np.mean(np.abs(predictions - y_test) < 0.5)

    metrics = {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'Accuracy': accuracy
    }

    return metrics, predictions



In [ ]:
# Función para buscar los mejores hiperparámetros
def hyperparameter_tuning(X_train, y_train, X_val, y_val, input_channels, sequence_length, num_features):

    epochs_list = [100, 300, 500]
    learning_rates = [0.01, 0.001]
    optimizers = ['Adam', 'SGD', 'RMSprop']

    best_val_loss = float('inf')
    best_model = None
    best_params = {}

    results = []

    for epochs in epochs_list:
        for lr in learning_rates:
            for opt_name in optimizers:
                print(f"\nProbando con: Épocas={epochs}, LR={lr}, Optimizador={opt_name}")

                # Inicializar modelo
                model = DengueForecastCNN(input_channels, sequence_length, num_features)

                # Seleccionar optimizador
                if opt_name == 'Adam':
                    optimizer = optim.Adam(model.parameters(), lr=lr)
                elif opt_name == 'SGD':
                    optimizer = optim.SGD(model.parameters(), lr=lr)
                else:  # RMSprop
                    optimizer = optim.RMSprop(model.parameters(), lr=lr)

                # Función de pérdida
                criterion = nn.MSELoss()

                # Entrenar modelo
                model, train_losses, val_losses = entrenar_modelo(
                    X_train, y_train, X_val, y_val,
                    model, optimizer, criterion, epochs, batch_size=32
                )

                # Evaluar en validación
                final_val_loss = val_losses[-1]

                # Guardar resultados
                results.append({
                    'epochs': epochs,
                    'learning_rate': lr,
                    'optimizer': opt_name,
                    'val_loss': final_val_loss
                })

                # Actualizar mejor modelo si es necesario
                if final_val_loss < best_val_loss:
                    best_val_loss = final_val_loss
                    best_model = model
                    best_params = {
                        'epochs': epochs,
                        'learning_rate': lr,
                        'optimizer': opt_name
                    }

    # Mostrar resultados de la búsqueda
    results_df = pd.DataFrame(results)
    print("\nResultados de la búsqueda de hiperparámetros:")
    print(results_df.sort_values('val_loss'))

    print(f"\nMejores hiperparámetros encontrados:")
    print(f"Épocas: {best_params['epochs']}")
    print(f"Learning Rate: {best_params['learning_rate']}")
    print(f"Optimizador: {best_params['optimizer']}")
    print(f"Pérdida de validación: {best_val_loss:.4f}")

    return best_model, best_params



In [ ]:
# Definir listas de hyperparámetros
epochs_list = [100, 300, 500]
learning_rates = [0.01, 0.001]
optimizers = [optim.Adam, optim.AdamW, optim.SGD, optim.RMSprop]

#epochs_list = [100]
#learning_rates = [0.01]
#optimizers = [optim.Adam]


In [ ]:
# Parámetros para las secuencias
sequence_length = 12  # 12 semanas de historial
forecast_horizon = 3  # Pronóstico a 3 semanas

# Preparar secuencias
X, y, feature_cols = preparar_sequences(train_df, sequence_length, forecast_horizon)

# Dividir en entrenamiento y validación (80/20)
split_idx = int(len(X) * 0.8)
X_train, X_val = X[:split_idx], X[split_idx:]
y_train, y_val = y[:split_idx], y[split_idx:]

print(f"Forma de los datos de entrenamiento: {X_train.shape}")
print(f"Forma de los datos de validación: {X_val.shape}")

# Parámetros del modelo
input_channels = X_train.shape[2]  # Número de características
num_features = X_train.shape[1]    # Longitud de la secuencia

# Búsqueda de hiperparámetros
best_model, best_params = hyperparameter_tuning(
  X_train, y_train, X_val, y_val,
  input_channels, sequence_length, num_features
  )

# Evaluar el mejor modelo
metrics, _ = evaluate_model(best_model, X_val, y_val)

print("\nResumen del modelo:")
print(best_model)

print("\nMétricas en el conjunto de validación:")
for metric, value in metrics.items():
  print(f"{metric}: {value:.4f}")

  # Preparar datos de prueba
  X_test, _, _ = prepare_sequences(test_df, sequence_length, forecast_horizon)

  # Hacer predicciones en datos de prueba
  best_model.eval()
  X_test_tensor = torch.FloatTensor(X_test)
  with torch.no_grad():
    test_predictions = best_model(X_test_tensor).numpy()

  print("\nSe han generado predicciones para los datos de prueba.")
  print(f"Forma de las predicciones: {test_predictions.shape}")

  # Si hay un archivo de envío, guardar las predicciones
  if "SUBMISSION_DIR" in config:
    submission_df = pd.read_csv(config["SUBMISSION_DIR"])
    submission_df['dengue'] = test_predictions
    submission_df.to_csv('submission_predicted.csv', index=False)
    print("Predicciones guardadas en 'submission_predicted.csv'")